In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import h5py
import os

sns.set_theme('poster')

def GetTemplateFiles(config):
    template_path = '/storage/home/adz6/group/project/datasets/data/dense_template_grid'
    
    radii, angles = np.meshgrid(config['r'], config['pa'])
    
    template_list = []
    for i, param_grp in enumerate(zip(radii.flatten(), angles.flatten())):
        for j, template_file in enumerate(os.listdir(template_path)):
            #scores_name = score_path.split('/')[-1]

            angle = template_file.split('grid_')[-1].split('_')[0]
            rad = template_file.split('cm.h5')[0].split('_')[-1]
            
            if (param_grp[0] == int(rad)) and (param_grp[1] == float(angle)):
                #print(template_file)
                template_list.append(os.path.join(template_path, template_file))

    return template_list

config = {
    'r': [1,],
    'pa': [86.0,87.0, 88.0],
}

In [ ]:

template_file_paths = GetTemplateFiles(config)

In [ ]:
template_file_paths

In [ ]:
templates = h5py.File(template_file_paths[2], 'r')

In [ ]:
templates['data'].shape

# select a random template signal, reshape

In [ ]:
rng = np.random.default_rng()
rand_int = rng.integers(0, templates['data'].shape[0], 1)

template = templates['data'][rand_int, :]

template = template.reshape(60, template.shape[-1] // 60)

signal = np.copy(template)

# select a template length and normalize template

In [ ]:
var = 1.38e-23 * 10 * 50 * 200e6 
length = 8192 * 1

norm = 1 / np.sqrt(var * np.vdot(template[:, 0:length].flatten(), template[:, 0:length].flatten()))
template = norm * template[:, 0:length]

# for a range of signal lengths, calculate the score/match ratio


In [ ]:
signal_lengths = np.int32(np.logspace(5, 14.5, 51, base=2))

In [ ]:
signal_lengths

# if we assume that the start conditions are the same but the signal ends at a different time than the template

In [ ]:
score_array = []
ideal_score_array = []

for i, sig_length in enumerate(signal_lengths):
    
    signal_array = np.zeros(template.shape, dtype=np.complex64)
    
    if sig_length <= length:
    
        signal_array[:, 0:sig_length] = signal[:, 0:sig_length]
        
    else:
        signal_array[:, :] = signal[:, 0:length]
    
    score = abs(signal_array.conjugate() * template).sum()
    
    score_array.append(score)
    
    norm = abs( 1 / np.sqrt(var * np.vdot(signal[:, 0:sig_length].flatten(), signal[:, 0:sig_length].flatten())))
    
    ideal_score = norm * abs(signal[:, 0:sig_length].conjugate() * signal[:, 0:sig_length]).sum()
    
    ideal_score_array.append(ideal_score)
    
score_array = np.array(score_array)
ideal_score_array = np.array(ideal_score_array)

In [ ]:
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
colors = sns.color_palette('deep', 10)

ax.plot(signal_lengths, score_array, '-', label = 'Score with Fixed Template Length')
ax.plot(signal_lengths, ideal_score_array, '-', label='Ideal Score')

ax.legend(loc=4)
ax.set_title(f'MF Score for Fixed Template Length = {length}')
ax.set_ylabel('Score')
ax.set_xlabel('Signal Length (Samples))')

In [ ]:
#signal_lengths = np.int32(np.logspace(5, 14, 51, base=2))

x1 = np.linspace(1, length, 101)
x2 = np.linspace(length, 3 * 8192, 101)

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)

ax.plot(signal_lengths, score_array / ideal_score_array, '.')
ax.plot(x1, np.sqrt(x1 / length), )
ax.plot(x2,  np.sqrt(length / x2 ))

ax.set_title(f'Match for Fixed Template Size = {length} Samples')
ax.set_ylabel('Match')
ax.set_xlabel('Signal Length (Samples)')

# now assume that the signal starts during the template

In [ ]:
start_time_diff = np.int32(np.linspace(0, length / 2, 201))

In [ ]:
template = templates['data'][rand_int, :]

template = template.reshape(60, template.shape[-1] // 60)

signal = np.copy(template)

var = 1.38e-23 * 10 * 50 * 200e6 
#length = 2 * 8192

norm = 1 / np.sqrt(var * np.vdot(template[:, 0:length].flatten(), template[:, 0:length].flatten()))
template = norm * template[:, 0:length]

In [ ]:
score_array_time = []
ideal_score_array = []

for i, diff in enumerate(start_time_diff):
    
    signal_array = np.zeros(template.shape, dtype=np.complex64)
    
    if diff >= 0:
        signal_array[:, diff:] = signal[:, 0:length-diff]

    score = abs(signal_array.conjugate() * template).sum()
    
    score_array_time.append(score)
    
    norm = abs( 1 / np.sqrt(var * np.vdot(signal[:, 0:length-diff].flatten(), signal[:, 0:length-diff].flatten())))
    
    ideal_score = norm * abs(signal[:, 0:length-diff].conjugate() * signal[:, 0:length-diff]).sum()
    
    ideal_score_array.append(ideal_score)
    
score_array_time = np.array(score_array_time)
ideal_score_array = np.array(ideal_score_array)

In [ ]:
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
sns.set_theme(style='ticks', context='poster')
colors = sns.color_palette('deep', 10)

ax.plot(start_time_diff, score_array_time, '-', label = 'Score with Start Time Mismatch')
ax.plot(start_time_diff, ideal_score_array, '-', label = 'Score with Identical Start Time')

ax.set_xlabel('Start Time Offset (Samples)')
ax.set_ylabel('Score')
ax.legend(loc=1)

# the signal starts during the template but we multiply the fft

In [ ]:
templates = h5py.File(template_file_paths[2], 'r')

rand_int = rng.integers(0, templates['data'].shape[0], 1)
template = templates['data'][rand_int, :]
var = 1.38e-23 * 10 * 50 * 200e6  / length
#length = 8192

template = template.reshape(60, template.shape[-1] // 60)[:, 0:length]
signal = np.copy(template)

template = np.fft.fft(template, axis = -1) / (length)


In [ ]:
#var = 1.38e-23 * 10 * 50 * 200e6  / 8192
#length = 8192

norm = 1 / np.sqrt(var * np.vdot(template.flatten(), template.flatten()))
template = norm * template

In [ ]:
score_array = []
ideal_score_array = []

for i, diff in enumerate(start_time_diff):
    
    signal_array = np.zeros(template.shape, dtype=np.complex64)
    
    signal_array[:, diff:] = signal[:, 0:length-diff]
    
    signal_array = np.fft.fft(np.exp(-1j * np.pi/2) * signal_array, axis=-1) / length

    score = abs(signal_array.conjugate() * template).sum()
    
    score_array.append(score)
    
    norm = abs( 1 / np.sqrt(length * var * np.vdot(signal[:, 0:length-diff].flatten(), signal[:, 0:length-diff].flatten())))
    
    ideal_score = norm * abs(signal[:, 0:length-diff].conjugate() * signal[:, 0:length-diff]).sum()
    
    ideal_score_array.append(ideal_score)
    
score_array = np.array(score_array)
ideal_score_array = np.array(ideal_score_array)

In [ ]:
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
colors = sns.color_palette('deep', 10)

ax.plot(start_time_diff, score_array, '-', label = f'Score with Start Time Mismatch, Freq.')
ax.plot(start_time_diff, score_array_time, '-', label = f'Score with Start Time Mismatch, Time')
ax.plot(start_time_diff, ideal_score_array, '-', label = f'Score with Identical Start Time')

ax.set_xlabel(r'Start Time Offset, $\Delta N$ (Samples)')
ax.set_ylabel('Score')


#ax.plot(start_time_diff, ideal_score_array[0] * ((length - start_time_diff ) / length) ** (1/2), label=r'$\left(\frac{N_T-\Delta N}{N_T}\right)^{\frac{1}{2}}$')
#arb_power = (2 / 3)
#ax.plot(start_time_diff, ideal_score_array[0] * ((length - start_time_diff ) / length) ** (arb_power), label=r'$\left(\frac{N_T-\Delta N}{N_T}\right)^{\frac{2}{3}}$')

ax.legend(loc=(0.3, 1.01))

In [ ]:
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
colors = sns.color_palette('deep', 10)

#ax.plot(start_time_diff, score_array, '.')
ax.plot(start_time_diff, score_array / ideal_score_array, '.', label='Match')


ax.set_xlabel(r'Start Time Offset, $\Delta N$ (Samples)')
ax.set_ylabel('Match')


arb_power1 = (1 / 6.5)
arb_power2 = (1 / 6)
arb_power3 = (1 / 3)

x = start_time_diff
y = 1 * ((length - start_time_diff ) / length) ** (arb_power1)
#y = 0.333 * ((8192 - start_time_diff ) / 8192) ** (arb_power1) + 0.333 * ((8192 - start_time_diff ) / 8192) ** (arb_power2) + 0.333 * ((8192 - start_time_diff ) / 8192) ** (arb_power3)
ax.plot(x, y, label=r'$\left(\frac{N_T-\Delta N}{N_T}\right)^{\frac{1}{6.5}}$')
ax.legend(loc=3)

In [ ]:
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
colors = sns.color_palette('deep', 10)

#ax.plot(start_time_diff, score_array, '.')
ax.plot(start_time_diff, score_array / ideal_score_array, '.', label='Match')


ax.set_xlabel(r'Start Time Offset, $\Delta N$ (Samples)')
ax.set_ylabel('Match')


arb_slope = (- 0.2 / length )
arb_power2 = (1 / 6)
arb_power3 = (1 / 3)

x = start_time_diff 
y = 1 + arb_slope * x
#y = 0.333 * ((8192 - start_time_diff ) / 8192) ** (arb_power1) + 0.333 * ((8192 - start_time_diff ) / 8192) ** (arb_power2) + 0.333 * ((8192 - start_time_diff ) / 8192) ** (arb_power3)
ax.plot(x, y, label=r'$\frac{-\Delta N}{5\times N_T}$')
ax.legend(loc=3)

# vary both the signal start time and the lengths of the signals

In [ ]:
ngrid = 51

start_time_diff = np.int32(np.linspace(-8191, 8191, ngrid))
signal_lengths = np.int32(np.linspace(1, 2 * 8192, ngrid))

#start_time_diff = [-100, 0, 100]
#signal_lengths = [10, 20, 50]

score_array = np.zeros((ngrid, ngrid))
ideal_score_array = np.zeros((ngrid, ngrid))

var = 1.38e-23 * 10 * 50 * 200e6  / 8192
length = 8192

for i, diff in enumerate(start_time_diff):
    
    for j, sig_length in enumerate(signal_lengths):
        
        template = templates['data'][rand_int, :]
        
        # define signal and template
        signal = template.reshape(60, template.shape[-1] // 60)
        template = template.reshape(60, template.shape[-1] // 60)[:, 0:length]

        # fft and normalize template
        template = np.fft.fft(template, axis = -1) / (length)
        norm = 1 / np.sqrt(var * np.vdot(template.flatten(), template.flatten()))
        template = norm * template
        
        signal_array = np.zeros(template.shape, dtype=np.complex64)
        
        if diff >= 0:
    
            # the signal starts after the template starts
            signal_array[:, diff:] = signal[:, 0:length-diff]
            
            # the signal ends before the template ends
            if sig_length < (length-diff):
                signal_array[:, sig_length + diff:] = 0

        elif diff < 0:
            
            # the signal starts before the template starts
            signal_array[:, :] = signal[:, -diff:length-diff]
            
            # the signal ends before the template starts
            if sig_length < -diff:
                signal_array[:, :] = 0
            elif sig_length < (length-diff): # the signal ends sometime during the template
                signal_array[:, sig_length + diff:] = 0
                
                
        #plt.figure()
        #plt.plot(signal_array.real[0, :])
        
        #plt.xlim(0, 400)
        #plt.title(f'{diff}_{sig_length}')
        #plt.show()
        

        signal_array = np.fft.fft(signal_array, axis=-1) / length
        
        score = abs(signal_array.conjugate() * template).sum()
    
        score_array[i, j] = score
        
        if score == 0:
            ideal_score_array[i, j] = 0
        else:
            norm = abs( 1 / np.sqrt(var * np.vdot(signal_array.flatten(), signal_array.flatten())))
            ideal_score = norm * abs(signal_array.flatten().conjugate() * signal_array.flatten()).sum()
            ideal_score_array[i, j] = ideal_score
            
            

In [ ]:
sns.set_theme(style='ticks')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
cmap = sns.color_palette('mako_r', as_cmap=True)

img = ax.imshow(
    score_array, 
    cmap=cmap, 
    aspect='auto', 
    extent = (signal_lengths[0], signal_lengths[-1], start_time_diff[-1], start_time_diff[0]),
)
cbar = fig.colorbar(img, label='Score')
ax.set_xlabel('Signal Length (Samples)')
ax.set_ylabel('Signal Start Relative to Template (Samples)')


In [ ]:
sns.set_theme(style='ticks')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
cmap = sns.color_palette('mako_r', as_cmap=True)

img = ax.imshow(
    ideal_score_array, 
    cmap=cmap, 
    aspect='auto', 
    extent = (signal_lengths[0], signal_lengths[-1], start_time_diff[-1], start_time_diff[0]),
)
cbar = fig.colorbar(img, label='Score')
ax.set_xlabel('Signal Length (Samples)')
ax.set_ylabel('Signal Start Relative to Template (Samples)')


In [ ]:
sns.set_theme(style='ticks')
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
cmap = sns.color_palette('mako_r', as_cmap=True)

match = score_array / ideal_score_array
match_isnan = np.isnan(match)
match[match_isnan] = 0

img = ax.imshow(
    match, 
    cmap=cmap, 
    aspect='auto', 
    extent = (signal_lengths[0], signal_lengths[-1], start_time_diff[-1], start_time_diff[0]),
)
cbar = fig.colorbar(img, label='Match')
ax.set_xlabel('Signal Length (Samples)')
ax.set_ylabel('Signal Start Relative to Template (Samples)')